#. 🏦 Banking Service Agent 💵

**What's about:** Banking Customer Service Agent with RAG and Sqlite tools for products and policy related questions.

Dev by Jeison Robles.

In [ ]:
%pip install -q openai qdrant-client docling fastembed jupyter-chat-widget

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.0/304.0 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 3.0 MB/s eta 0:00:00
 

In [ ]:
from IPython.display import display, HTML

def show_space_banner():
    display(HTML(r"""
<pre style="
  background: radial-gradient(circle at 30% 20%, #0b1020 0%, #05070f 45%, #020617 100%);
  color: #e5e7eb;
  padding: 18px 20px;
  border-radius: 14px;
  font-family: ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, 'Liberation Mono', monospace;
  font-size: 13px;
  line-height: 1.25;
  box-shadow: 0 18px 40px rgba(0,0,0,.55);
  border: 1px solid rgba(255,255,255,.08);
  overflow-x: auto;
">
<span style="color:#22d3ee;">✦</span>     <span style="color:#a78bfa;">✦</span>        <span style="color:#22c55e;">✦</span>       <span style="color:#fbbf24;">✦</span>          <span style="color:#22d3ee;">✦</span>
     <span style="color:#94a3b8;">·</span>     <span style="color:#94a3b8;">·</span>     <span style="color:#94a3b8;">·</span>        <span style="color:#94a3b8;">·</span>     <span style="color:#94a3b8;">·</span>            <span style="color:#94a3b8;">·</span>

                    <span style="color:#38bdf8;">🚀</span>
                    <span style="color:#94a3b8;">/\\</span>
                   <span style="color:#94a3b8;">/  \\</span>
                  <span style="color:#94a3b8;">/____\\</span>
                  <span style="color:#94a3b8;">|    |</span>
                  <span style="color:#94a3b8;">|BANK|</span>
                  <span style="color:#94a3b8;">|AGNT|</span>
                  <span style="color:#94a3b8;">|____|</span>
                 <span style="color:#f97316;">/\\</span>  <span style="color:#f97316;">/\\</span>
                <span style="color:#f59e0b;">/  \\</span><span style="color:#f59e0b;">/  \\</span>
                  <span style="color:#fbbf24;">||</span>  <span style="color:#fbbf24;">||</span>
                   <span style="color:#fbbf24;">'</span>  <span style="color:#fbbf24;">'</span>

<span style="color:#22d3ee;">────────────────────────────────────────────────────────────</span>
<span style="color:#e5e7eb; font-weight:700;">SPACE OPS · Banking Service Agent· Jeison Robles</span>
<span style="color:#94a3b8;">Policy-aware · RAG · Tools · Safe-by-default</span>

<span style="color:#22c55e;">STATUS:</span> <span style="color:#22d3ee;">ONLINE</span>    <span style="color:#a78bfa;">MODE:</span> DEMO    <span style="color:#fbbf24;">LOG:</span> ENABLED
<span style="color:#94a3b8;">Tip:</span> Provide a <span style="color:#e5e7eb; font-weight:700;">Customer ID</span> only for product lookups.
</pre>
"""))

show_space_banner()

# Prep Functions

In [ ]:
import openai
import os
from google.colab import userdata      #API to access user secrets
from jupyter_chat_widget import ChatUI
from time import sleep


def get_oai_client():
    """
        Creates the OpenAI client.
    """
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    client = openai.OpenAI()
    messages = [{"role": "system",
                 "content": """
                 You are a helpful banking assistant that will provide easy to understand answers about the bank policy or related to customers products,
                 If the user asks about my products/accounts and provides a customer_id, call get_customer_name and get_customer_products.
                 When customer products are available, always respond using this structure:

                 1) Greeting (with customer name if known)
                 2) Product summary (bullets). Include product_type, nickname, currency, balance, credit_limit, apr.
                 3) What the negative balance means (if any), in plain language.
                 4) What actions are allowed according to policy (cite from snippets).
                 Then summarize what products they have and answer using the policy with the search tool.
                 Output in HTML."""}]
    return client, messages


def query_llm(ui, client, messages):
    """
        Query the LLM and returns a stream handle to the response.
    """
    ui.rewrite("[Generating]")
    stream = client.chat.completions.create(
        model="gpt-5.2",  # Or another suitable model
        messages=messages, # Use the message history
        stream=True, # Enable streaming
    )
    return stream


def display_response(ui, stream):
    """
        Display the LLM's response in the UI one token at a time.
    """
    ui.rewrite("")
    complete_response = ""
    for chunk in stream:
        if (content := chunk.choices[0].delta.content) and content.strip():
            complete_response += content
            ui.append(content)
    return complete_response


# Import Policy

We'll download a sample bank policy directly from the **GitHub Repo**

In [ ]:
import os
import requests

urls_and_filenames = [
    (
        "https://raw.githubusercontent.com/JeisonRobles/Banking-Service-Agent/main/Bank_Policy_1",
        "Bank_Policy_1.md"
    ),
]

os.makedirs("documents", exist_ok=True)

for url, filename in urls_and_filenames:
    response = requests.get(url)
    response.raise_for_status()
    with open(os.path.join("documents", filename), "wb") as f:
        f.write(response.content)

___
# Documents processing

In [ ]:
from docling.document_converter import DocumentConverter
from docling.chunking import HybridChunker
from docling.datamodel.base_models import InputFormat
from docling.document_converter import DocumentConverter
from uuid import uuid4
from qdrant_client import QdrantClient, models
paths = ["/content/documents/Bank_Policy_1.md"]

def ingest_documents(ui, paths):
    print("Ingesting documents...")
    print(paths)
    # Prepare the vectordb and embedder
    vdb = QdrantClient(location=":memory:")
    dense_model = "sentence-transformers/all-MiniLM-L6-v2"
    vdb.set_model(dense_model)
    collection_name = "documents"
    vdb.create_collection(
        collection_name=collection_name,
        vectors_config=vdb.get_fastembed_vector_params(),
    )
    points = []
    for path in paths:
        print(path)
    print(f"processing path: {paths}")
    ui.rewrite(f"[Parsing {paths}...]")

    for path in paths:

        print(f"processing path: {path}")

        ui.rewrite(f"[Parsing {path}...]")

        # Parse the document with docling
        doc = DocumentConverter().convert(source=path).document

        # Chunk the document
        chunker = HybridChunker()
        chunk_iter = chunker.chunk(dl_doc=doc)

        # Enrich the chunks and build Qdrant points
        for chunk in chunk_iter:
            enriched_text = chunker.contextualize(chunk=chunk)
            meta = chunk.meta.export_json_dict()
            points.append(
                models.PointStruct(
                    id=uuid4().hex,
                    payload=meta | {"document": enriched_text},
                    vector={
                        # FastEmbed uses named vector fields derived from the model names
                        vdb.get_vector_field_name(): models.Document(text=enriched_text, model=dense_model),
                    },
                )
            )
    vdb.upload_points(collection_name=collection_name, points=points, batch_size=64, wait=True)
    ui.rewrite(f"All documents were processed. I'm ready!")
    return vdb


#=========================================================
# Defining add_context tool
#=========================================================
def add_context(ui, vdb, query):
    """
        Queries the vector database for relevant context snippets
        and adds them to the LLM's context.
    """

    print("Adding context...")
    ui.rewrite("[Searching]")
    samples = vdb.query(
        collection_name="documents",
        query_text=query,
        limit=10,
    )
    ui.rewrite(f"[Found {len(samples)} relevant snippets]")
    sleep(1)
    return {
        "role": "user",
        "content": f"Relevant snippets from the document: {'\n\n'.join(s.document for s in samples)}"
    }

___
## 💾 SQL Local DB

Lets implement an small SQLite db for customers patterns.

This will has just 2 related tables:
1. customers: With customer ID and the Full Name
2. products:  Contains the products and aggregated dummy information about each product (simulating some Analytical ETL view)


In [ ]:
import sqlite3
from pathlib import Path

DB_PATH = Path("bank_demo.sqlite")

def init_customer_db(db_path: Path = DB_PATH):
    conn = sqlite3.connect(str(db_path))
    conn.execute("PRAGMA foreign_keys = ON;")
    cur = conn.cursor()

    # 1) Customers
    cur.execute("""
    CREATE TABLE IF NOT EXISTS customers (
      customer_id TEXT PRIMARY KEY,
      full_name   TEXT NOT NULL
    )
    """)

    # 2) Products (linked to customers)
    cur.execute("""
    CREATE TABLE IF NOT EXISTS products (
      product_id   TEXT PRIMARY KEY,
      customer_id  TEXT NOT NULL,
      product_type TEXT NOT NULL,
      nickname     TEXT,
      currency     TEXT,
      balance      REAL,
      credit_limit REAL,
      apr          REAL,
      FOREIGN KEY(customer_id) REFERENCES customers(customer_id)
    )
    """)

    # Seed customers (idempotent)
    cur.executemany(
        "INSERT OR IGNORE INTO customers (customer_id, full_name) VALUES (?, ?)",
        [
            ("CUST-1001", "Alicia Mora"),
            ("CUST-2002", "Diego Hernández"),
            ("CUST-3003", "María José Vargas"),
        ],
    )

    # Seed products (idempotent)
    cur.executemany(
        """
        INSERT OR IGNORE INTO products
        (product_id, customer_id, product_type, nickname, currency, balance, credit_limit, apr)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """,
        [
            ("P-CHK-01", "CUST-1001", "checking",       "Daily Checking",   "USD",  1250.75,   None,     None),
            ("P-CC-01",  "CUST-1001", "credit_card",    "Travel Card",      "USD",  -230.10,  5000.00,  0.279),
            ("P-SAV-01", "CUST-2002", "savings",        "Business Savings", "USD", 18250.00,   None,     None),
            ("P-LOC-01", "CUST-2002", "line_of_credit", "Working Capital",  "USD", -1200.00, 25000.00,  0.185),
        ],
    )

    conn.commit()
    conn.close()
    return str(db_path.resolve())

DB_RESOLVED = init_customer_db()
print(f"SQLite demo DB ready at: {DB_RESOLVED}")

SQLite demo DB ready at: /content/bank_demo.sqlite


In [ ]:

# --- Tool: lookup customer name (used only for "my products"/personal info requests) ---
import sqlite3
from typing import Dict, Any
import json


#=========================================================
# Defining get_customer_name tool
#=========================================================
def get_customer_name(customer_id: str) -> Dict[str, Any]:
    """Return customer name for greeting; no products yet."""
    conn = sqlite3.connect(str(DB_PATH))
    cur = conn.cursor()
    cur.execute("SELECT full_name FROM customers WHERE customer_id = ?", (customer_id,))
    row = cur.fetchone()
    conn.close()
    if not row:
        return {"found": False, "customer_id": customer_id, "full_name": None}
    return {"found": True, "customer_id": customer_id, "full_name": row[0]}

# Tool implementations registry (keep/extend your existing one if present)
try:
    tool_implementations
except NameError:
    tool_implementations = {}


#Define tool implementations:
tool_implementations["get_customer_name"] = lambda ui, vdb, customer_id: {
    "role": "tool",
    "name": "get_customer_name",
    "content": json.dumps(get_customer_name(customer_id), ensure_ascii=False),
}



#=========================================================
# Defining get_customer_products tool
#=========================================================
import sqlite3
from typing import Dict, Any, List
import json

def get_customer_products(customer_id: str) -> Dict[str, Any]:
    conn = sqlite3.connect(str(DB_PATH))
    cur = conn.cursor()
    cur.execute("""
        SELECT product_id, product_type, nickname, currency, balance, credit_limit, apr
        FROM products
        WHERE customer_id = ?
        ORDER BY product_type
    """, (customer_id,))
    rows = cur.fetchall()
    conn.close()

    products: List[Dict[str, Any]] = []
    for r in rows:
        products.append({
            "product_id": r[0],
            "product_type": r[1],
            "nickname": r[2],
            "currency": r[3],
            "balance": r[4],
            "credit_limit": r[5],
            "apr": r[6],
        })

    return {
        "customer_id": customer_id,
        "count": len(products),
        "products": products
    }

# register tool implementation (DO NOT overwrite tool_implementations)
tool_implementations.update({
    "get_customer_products": lambda ui, vdb, customer_id: {
        "role": "tool",
        "name": "get_customer_products",
        "content": json.dumps(get_customer_products(customer_id), ensure_ascii=False),
    }
})


In [ ]:
import json

def _invoke_tool(ui, vdb, tool_call):
    tool_name = tool_call.function.name
    ui.rewrite(f"Calling tool {tool_name}")

    tool_args = json.loads(tool_call.function.arguments or "{}")

    tool_impl = tool_implementations[tool_name]
    output = tool_impl(ui, vdb, **tool_args)  # returns {"role":"tool","name":...,"content":...}

    # IMPORTANT: tool message MUST include tool_call_id
    tool_msg = {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": output["content"],
    }
    return tool_msg


def query_llm_with_tools(ui, client, vdb, messages, tools, max_rounds=5):       # Agent Loop
    """
    Robust minimal Agent loop:
    - Keep resolving tool calls until the model returns a normal assistant message (no tool_calls)
    - Then stream the final answer
    """
    for _ in range(max_rounds):
        ui.rewrite("[Thinking...]")

        resp = client.chat.completions.create(
            model="gpt-5.2",
            messages=messages,
            tools=tools,
            tool_choice="auto",
            stream=False,
        )

        assistant_msg = resp.choices[0].message
        tool_calls = getattr(assistant_msg, "tool_calls", None)

        # If the model is NOT requesting tools, stream final answer
        if not tool_calls:

            print("Did not chain Tools...")

            ui.rewrite("Did not chain Tools...")
            ui.rewrite("[Generating]")

            stream = client.chat.completions.create(
                model="gpt-5.2",
                messages=messages,
                stream=True,
            )
            return stream

        # Append assistant message WITH tool_calls (required)
        messages.append({
            "role": "assistant",
            "content": assistant_msg.content,
            "tool_calls": [tc.model_dump() for tc in tool_calls],
        })
        ui.rewrite("Calling Tools...")
        print("Calling Tools...")
        # Execute tools and append tool results
        for tc in tool_calls:
            tool_msg = _invoke_tool(ui, vdb, tc)
            messages.append(tool_msg)
    # If we hit max rounds, fail safely
    ui.rewrite("Agent Loop Ended...")
    print("Agent Loop Ended...")
    return client.chat.completions.create(
        model="gpt-5.2",
        messages=messages + [{"role": "user", "content": "Please answer without calling more tools."}],
        stream=True,
    )

In [ ]:
# keep existing tools and add/override entries
tool_implementations.update({
    "search": add_context,
    "get_customer_name": lambda ui, vdb, customer_id: {
        "role": "tool",
        "name": "get_customer_name",
        "content": json.dumps(get_customer_name(customer_id), ensure_ascii=False),
    }
})


#=========================================================
# Set search schema
#=========================================================
search = {
    "type": "function",
    "function": {
        "name": "search",
        "description": "Use this tool when the user is asking you something you don't know. You should use this tool very often and you can call it many times in a row if necessary.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description":
                        "The query to retrieve the information from the document store using pure embedding similarity search",
                },
            },
            "required": ["query"],
        },
    }
}


#=========================================================
# Set get_customer_name schema
#=========================================================
get_customer_name_tool = {
  "type": "function",
  "function": {
    "name": "get_customer_name",
    "description": "Look up the customer's full name by customer_id for a personalized greeting. Use only when the user asks about their own products/accounts and provides a customer_id.",
    "parameters": {
      "type": "object",
      "properties": {
        "customer_id": {
          "type": "string",
          "description": "Customer ID in format CUST-#### (e.g., CUST-2002)."
        }
      },
      "required": ["customer_id"]
    }
  }
}

#=========================================================
# Set get_customer_products schema
#=========================================================
get_customer_products_tool = {
  "type": "function",
  "function": {
    "name": "get_customer_products",
    "description": "Fetch the list of products for a customer_id. Use only when the user asks about their own products/accounts and provides a customer_id.",
    "parameters": {
      "type": "object",
      "properties": {
        "customer_id": {
          "type": "string",
          "description": "Customer ID in format CUST-#### (e.g., CUST-2002)."
        }
      },
      "required": ["customer_id"]
    }
  }
}



def Entry_Point(paths = paths):
    show_space_banner()                                                       # Shows the banner
    ui = ChatUI()                                                             # Starts the UI Chat
    vdb = ingest_documents(ui, paths)                                         # Ingest the policy (RAG)
    client, messages = get_oai_client()                                       # Starts the OAI Client

    def Agent_Loop(query):
        messages.append({"role": "user", "content": query})                   #History
        #print(query)
        stream = query_llm_with_tools(                                        # Call LLM and pass tools and memory
            ui, client, vdb, messages,
            [search, get_customer_name_tool, get_customer_products_tool]      # Passing the schemas
        )
        response = display_response(ui, stream)
        messages.append({"role": "assistant", "content": response})

    ui.connect(lambda query: Agent_Loop(query))



In [ ]:
Entry_Point()

Output()

Output()

Text(value='', description='user: ')

Ingesting documents...
['/content/documents/Bank_Policy_1.md']


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Did not chain Tools...
Calling Tools...
Calling Tools...
Adding context...


/usr/local/lib/python3.12/dist-packages/qdrant_client/common/client_warnings.py:7: UserWarning: `query` method has been deprecated and will be removed in 1.17. Instead, inference can be done internally within regular methods like `query_points` by wrapping data into `models.Document` or `models.Image`.
  warnings.warn(message, category, stacklevel=stacklevel)


Did not chain Tools...
Did not chain Tools...
Calling Tools...
Adding context...
Did not chain Tools...
